# Ejercicio Stream Cipher

In [1]:
import random

In [3]:
def decimalToBinary(decimal, bits=8):
    binary = ''
    while decimal > 0:
        binary = str(decimal % 2) + binary
        decimal = decimal // 2
    
    binary = binary.rjust(bits, '0')
    return binary

def ascciToBinary(ascci):
    return ' '.join(decimalToBinary(ord(char)) for char in ascci)

def binaryToAscii(binary):
    binary = binary.replace(" ", "")
    return ''.join(chr(int(binary[i:i + 8], 2)) for i in range(0, len(binary), 8))

def xor(text, key):
    #text = cleanText(text)
    #key = cleanText(key)

    if len(text) > len(key):
        key = key * (len(text) // len(key)) + key[:len(text) % len(key)]
    elif len(text) < len(key):
        key = key[:len(text)]
        
    bin1 = ascciToBinary(text).replace(" ", "")
    bin2 = ascciToBinary(key).replace(" ", "")

    res = ""
    for i in range(len(bin1)):
        if bin1[i] == bin2[i]:
            res += "0"
        else:
            res += "1"
    return res

# 1. Generación de Keystream

## Un generador de números pseudoaleatorios (PRNG) básico

In [10]:
def keystreamGenerator(lenght, seed):
    random.seed(seed)
    return "".join(str(random.randint(32, 126)) for _ in range(lenght))

# 2. Cifrado

In [12]:
def encrypt(mensaje, keystream):
    return binaryToAscii(xor(mensaje, keystream))

# 3. Descifrado

In [6]:
def decryption(cipher, keystream):
    return binaryToAscii(xor(cipher, keystream))

# 4. Preguntas a Responder

## ¿Qué sucede cuando cambias la clave utilizada para generar el keystream?

Si se cambia la clave utilizada para generar el keystream, el mensaje cambiaría por completo. Obviamente, ya que al estar aplicando un XOR con otra llave el mensaje tomará un valor incorrecto. Si tengo el mensaje _m_ y la llave _k1_ con la que cifro _m_ obtengo _c1_; ahora, si intento descifrar _c1_ con _k1_, resulta en _d1_ el cual obviamente es igual a _m_, pero si lo hago con otra llave _k2_ el _d2_ obtenido NO será igual que _m_. Ya que la llave no fue correcta a la hora de cifrar.

In [18]:
m = "Hola mundo"
seed1 = "llave1"
seed2 = "llave2"

k1 = keystreamGenerator(len(m), seed1)
k2 = keystreamGenerator(len(m), seed2)

c1 = encrypt(m, k1)
d1 = decryption(c1, k1)
d2 = decryption(c1, k2) # cambio de llave, no será igual a m

print("m:", m)
print("c1:", c1)
print("m1:", d1)
print("m2:", d2)

m: Hola mundo
c1: y_YP_@\U^
m1: Hola mundo
m2: Ajni%jtnbn


## ¿Qué riesgos de seguridad existen si reutilizas el mismo keystream para cifrar dos mensajes diferentes?

El problema con utilizar el mismo keystream es que los cifrados obtenidos pueden llegar a ser analizados bajo un análisis de frecuencia, el cuál es más que suficiente para llevar a obtener el seed con el que el keystream fue generado.

## ¿Cómo afecta la longitud del keystream a la seguridad del cifrado?

La longitud debe de ser mínimo del mismo largo que el mensaje. Caso contrario, se tendrá que reutilizar parte del keystream, abriendo una vulnerabilidad. La cual sería que igual con un simple análisis de frecuencias el atacante pueda obtener información valiosa de este e incluso poder descifrar cuál fue el keystream utilizado si tiene más datos.

## ¿Qué consideraciones debes tener al generar un keystream en un entorno real?

Con asegurarse que se trate de la generación de un keystream TRNG, es un buen primer paso para que la integración al entorno real tenga una buena segurida. Obviamente también se debe tener en cuenta que el keystream solo se debe de utilizar una vez y debe de ser MÍNIMO del largo del mensaje. 

# Reflexión

Siempre es bueno recordar el hecho que los randoms que utilizamos cotidianamente NO son realmente randoms, son algoritmos pseudoaleatorios. Los cuales en algún punto se podrán obtener dada su repetición. Pero al momento de aplicar hechos externos, tales como el uso de sensores de temas ambientales o hardware se puede llegar a una solución más segura. Al tener una generación de randoms reales se puede tener un grado de certeza que el sistema es más seguro.

# Pruebas unitarias

In [ ]:
# Pruebas generadas con ChatGPT https://chatgpt.com/share/67bea404-3b44-8003-b930-9493c7ab41e5

import unittest

class TestKeystreamFunctions(unittest.TestCase):
    def test_encrypt_decrypt(self):
        mensaje = "Hello"
        seed = "llave"
        keystream = keystreamGenerator(len(mensaje), seed)
        cifrado = encrypt(mensaje, keystream)
        descifrado = decryption(cifrado, keystream)
        self.assertEqual(descifrado, mensaje)

    def test_encrypt_decrypt_with_error1(self):
        # Cambio de llave
        mensaje = "Hello"
        seed = "llave"
        keystream = keystreamGenerator(len(mensaje), seed)
        cifrado = encrypt(mensaje, keystream)
        wrongKeystream = keystreamGenerator(len(mensaje), "otrallave")
        descifrado = decryption(cifrado, wrongKeystream)
        self.assertEqual(descifrado, mensaje)

    def test_encrypt_decrypt_with_error2(self):
        mensaje = "Hello"
        seed = "llave"
        keystream = keystreamGenerator(len(mensaje), seed)
        cifrado = encrypt(mensaje, keystream)
        wrongKeystream = keystreamGenerator(len(mensaje) - 4, seed)
        descifrado = decryption(cifrado, wrongKeystream)
        self.assertEqual(descifrado, mensaje)
    
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.FF
FAIL: test_encrypt_decrypt_with_error1 (__main__.TestKeystreamFunctions)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\ealva\AppData\Local\Temp\ipykernel_14804\4011287499.py", line 20, in test_encrypt_decrypt_with_error1
    self.assertEqual(descifrado, mensaje)
AssertionError: 'Meohh' != 'Hello'
- Meohh
+ Hello


FAIL: test_encrypt_decrypt_with_error2 (__main__.TestKeystreamFunctions)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\ealva\AppData\Local\Temp\ipykernel_14804\4011287499.py", line 29, in test_encrypt_decrypt_with_error2
    self.assertEqual(descifrado, mensaje)
AssertionError: 'Heiom' != 'Hello'
- Heiom
+ Hello


----------------------------------------------------------------------
Ran 3 tests in 0.004s

FAILED (failures=2)
